# Detection of Relevant Object in Construction Site Pictures

This notebook demos [Amazon Rekognition Custom Labels](https://aws.amazon.com/rekognition/)' ability to learn how to detect relevant objects from pictures, using a dataset copied from [Context-based information generation from construction site images using unmanned aerial vehicle (UAV)-acquired data and image captioning](https://data.mendeley.com/datasets/4h68fmktwh/1). The dataset is licensed as [CC BY 4.0](http://creativecommons.org/licenses/by/4.0).

The notebook consists of the following steps:

1. Download and decompress the dataset
2. Load all image metadata and generate classes for each bounding box:
    1. Process the manifest file provided, consisting of the format explained on the [README](https://github.com/bobbruno/aws-rekognition-cl-demo) in the repository;
    2. Run the bounding boxes' descriptive phrases through [Amazon Comprehend](https://aws.amazon.com/comprehend/), to identify the syntactic elements.
    3. Extract the relevant elements from the description and process them (filtering irrelevant words/typos, lowercase, eliminating plurals) to be suitable object classes.
3. Split the data into training and test.
4. Generate an [Object Detection Ground Truth Manifest file](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/cd-required-fields.html) describing the dataset, which is required by Rekognition Custom Labels.
5. Create a Rekognition Project and Start the project version (model) training.

## Initial Setup/Configuration

In [26]:
import json
from PIL import Image as Imagem
import boto3
from pprint import pprint
from sklearn.model_selection import train_test_split

s3 = boto3.client("s3")
bucket = "sagemaker-us-east-1-113147044314"
input_path = "cl-demo/input"
reko_manifest_path = "cl-demo/reko-input"
output_path = "cl-demo/output"

## Dowload/Decompress the dataset

The dataset is actually stored in S3, so we just use the boto3 S3 interface to download it.

In [ ]:
s3.download_file("md-datasets-cache-zipfiles-prod", "4h68fmktwh-1.zip", "data/construction.zip")

In [ ]:
!unzip data/construction.zip -d data
!unzip data/image_with_captions.zip -d data
!rm data/construction.zip

## Generating Classifications from the Annotations

### Helper Classes for Processing the original manifest and generating object classifications

In [2]:
class BoundingBox(object):
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
    def __repr__(self):
        return str(self.__dict__)
    
class Token(object):
    def __init__(self, TokenId, Text, BeginOffset, EndOffset, Tag, Score):
        assert Score > 0.9, 'Not enough confidence on tag'
        self.TokenId = TokenId
        self.Text = Text
        self.LowerText = Text.lower()
        self.BeginOffset = BeginOffset
        self.EndOffset = EndOffset
        self.Tag = Tag
        self.Score = Score
    def __repr__(self):
        return str(self.__dict__)

        
class ImageAnnotation(object):
    COLORS = {'blue', 'red', 'yellow', 'green', 'white', 'silver', 'gray', 'grey', 'orange', 'black', 'brown', 'pink', 'purple', 'while'}
    STOPWORDS = {'elevens', 'fivr', 'plie', 'thirteens', 'pile', 'piles'}
    
    def __init__(self, index, bounding_box, phrase, comprehend=None):
        self.bb = bounding_box
        self.index = index
        self.phrase = phrase
        if (comprehend):
            self.update_tokens(comprehend)
        else:
            self.Tokens = None
            self.obj_class = None
        
    def _gen_tokens(self, comprehend):
        return(comprehend.detect_syntax(LanguageCode='en', Text=self.phrase)['SyntaxTokens'])
    
    def update_tokens(self, comprehend):
        self.Tokens = self._gen_tokens(comprehend) if len(self.phrase) else None
        if self.Tokens and len(self.Tokens):
            self.obj_class = self.get_class()
        else:
            self.obj_class = None
        
    def get_class(self):
        assert self.Tokens, "No tokenization available for class extraction"
        def get_object(tokens):
            noun_part = []
            for token in tokens:
                if token['PartOfSpeech']['Tag'] == 'VERB':
                    break
                if token['Text'].lower() in ImageAnnotation.COLORS:
                    continue
                if token['Text'].lower() in ImageAnnotation.STOPWORDS:
                    continue
                if token['PartOfSpeech']['Tag'] in ('ADJ', 'NOUN', 'PROPN'):
                    noun_part.append(token)
            return ' '.join([x['Text'] for x in noun_part])
        result = get_object(self.Tokens)
        if len(result) > 0:
            result = result[:-1] if result[-1] == 's' else result
        return(result)
        
    def __repr__(self):
        return str(self.__dict__)


class Image(object):
    def __init__(self, path, filename, size, annotations):
        self.path = path
        self.filename = filename
        self.size = size
        self.annotations = annotations
        im = Imagem.open(f"{self.path}/{self.filename}")
        self.width, self.height = im.size
        self.classify_image()
        
    def classify_image(self):
        self.obj_classes = {annotation.obj_class for annotation in self.annotations if annotation.obj_class}
    
    def gen_tokens(self, comprehend):
        for annot in self.annotations:
            annot.update_tokens(comprehend)
            
    def __repr__(self):
        return str(self.__dict__)


### Helper Function to process the manifest

In [3]:
def load_images(path, manifest_file, comprehend=None):
    with open(manifest_file, "r") as source_manifest:
        manifest = json.load(source_manifest)
    images = []
    for _, image in manifest.items():
        try:
            try:
                annotations = []
                for (index, annotation) in image['regions'].items():
                    shape_attrs = annotation['shape_attributes']
                    shape_attrs.pop('name')
                    phrase = annotation['region_attributes']['phrase']
                    img = ImageAnnotation(index, BoundingBox(**shape_attrs), phrase, comprehend)
                    if img.obj_class and len(img.obj_class):
                        annotations.append(img)
            except:
                pprint(annotation['shape_attributes'])
                raise
            images.append(
                Image(path, image['filename'], image['size'], annotations)
            )
        except:
            pprint(image)
            raise
    return(images)

In [4]:
comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')
image_objs = load_images("data/UAVdata", "data/via_region_data_final.json", comprehend)
[image.classify_image() for image in image_objs]

## Train/Test Split

In [12]:
def split_data(image_objs, pct_train=0.5, random_state=None):
    classified_images = [image for image in image_objs if len(image.obj_classes) > 0]
    unclassified_images = [image for image in image_objs if len(image.obj_classes) == 0]
    train_images, test_images = train_test_split(classified_images, train_size=pct_train, random_state=random_state)
    return(train_images, test_images, unclassified_images)

In [68]:
train_data, test_data, unclassified = split_data(image_objs, pct_train=0.2)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [70]:
len(test_data)

1130

## Preparing the Ground Truth Manifest

### Generate a mapping of the most common classes to ids (required by the manifest format)

In [21]:
from collections import Counter

class_map = {classif: i+1 for (i, (classif, _)) in
             enumerate(
                 Counter(
                     [an.obj_class for image in train_data for an in image.annotations if an.obj_class and len(an.obj_class)]
                 ).most_common(34)
             )}

In [128]:
def gen_manifest(image, class_map, s3_path, job_number=0, max_annotations=None, indent=None):
    img_annotations = image.annotations if max_annotations is None else image.annotations[:max_annotations]
    class_map['others'] = 0

    classes_in_img = {an.obj_class for an in img_annotations}
    annotations = [{
        "class_id": class_map.get(an.obj_class, 0),
        "left": an.bb.x,
        "top": an.bb.y,
        "width": an.bb.width,
        "height": an.bb.height
    } for an in img_annotations]
    
    manifest = {
        "source-ref": f'{s3_path}/{image.filename}',
        "bounding-box": { 
            "image_size": [
                {
                    "width": image.width,
                    "height": image.height,
                    "depth": 3 
                }
            ],
            "annotations": annotations
        },
        "bounding-box-metadata": { 
            "objects": [ {"confidence": 0.9} for _ in img_annotations ],
            "class-map": {v: k for k, v in class_map.items() if k in classes_in_img},
            "type": "groundtruth/object-detection",
            "human-annotated": "yes",
            "creation-date": "2020-02-25T22:50:05.0000z",
            "job-name": f"identify-construction-objs-{job_number}"
        }
     }
    return(json.dumps(manifest, indent=indent))

def save_manifest(data, path):
    s3_path = f"s3://{bucket}/{input_path}"
    with open(path, 'w') as outfile:
        for example in data:
            outfile.write(f"{gen_manifest(example, class_map, s3_path, max_annotations=5)}\n")

In [136]:
manifest_ext = "_5_limit_2"
print(f"Train Manifest name: s3://{bucket}/{reko_manifest_path}/train_manifest{manifest_ext}.json")
print(f"Test Manifest name: s3://{bucket}/{reko_manifest_path}/test_manifest{manifest_ext}.json")
#contruction-objects-train-max5

Train Manifest name: s3://sagemaker-us-east-1-113147044314/cl-demo/reko-input/train_manifest_5_limit_2.json
Test Manifest name: s3://sagemaker-us-east-1-113147044314/cl-demo/reko-input/test_manifest_5_limit_2.json


In [137]:
save_manifest(train_data, f'data/train_manifest{manifest_ext}.json')
save_manifest(test_data, f'data/test_manifest{manifest_ext}.json')

### Validating Manifest File

In [131]:
class Schemas:
    """
    Schema Definitions
    """

    SOURCE_SCHEMA = {
        "type": "object",
        "required": [
            "source-ref"
        ],
        "properties": {
            "source-ref": {
                "description": "Image S3 image URL",
                "type": "string",
                "pattern": "^s3://([^/]+)/(.*)$"
            }
        },
        "additionalProperties": False,
        'maxItems': 1,
        'minItems': 1
    }

    ATTRIBUTE_SCHEMA = {
        "definitions": {
            "image_size": {
                "type": "array",
                "items": {
                    "type": "object",
                    "required": [
                        "width",
                        "height",
                        "depth"
                    ],
                    "properties": {
                        "width": {
                            "$ref": "#/definitions/image_dimension"
                        },
                        "height": {
                            "$ref": "#/definitions/image_dimension"
                        },
                        "depth": {
                            "type": "number",
                            "min": 1,
                            "max": 3
                        }
                    }
                }
            },
            "annotations": {
                "type": "array",
                "items": {
                    "type": "object",
                    "required": [
                        "width",
                        "height",
                        "top",
                        "left",
                        "class_id"
                    ],
                    "properties": {
                        "width": {
                            "type": "number"
                        },
                        "height": {
                            "type": "number"
                        },
                        "top": {
                            "type": "number"
                        },
                        "left": {
                            "type": "number"
                        },
                        "class_id": {
                            "type": "integer"
                        }
                    }
                }
            },
            "image_dimension": {
                "type": "number",
                "minimum": 64,
                "maximum": 4096
            },
            "confidence": {
                "type": "number",
                "minimum": 0.0,
                "maximum": 1.0
            },
            "objects": {
                "type": "array",
                "items": {
                    "type": "object",
                    "required": [
                        "confidence"
                    ],
                    "properties": {
                        "confidence": {
                            "$ref": "#/definitions/confidence"
                        }
                    }
                }
            }
        },
        "patternProperties": {
            "^[A-Za-z0-9-]+": {
                "oneOf": [
                    {
                        "type": "object",
                        "description": "Detection label data",
                        "properties": {
                            "annotations": {
                                "$ref": "#/definitions/annotations"
                            },
                            "image_size": {
                                "$ref": "#/definitions/image_size"
                            }
                        },
                        "required": [
                            "annotations",
                            "image_size"
                        ]
                    },
                    {
                        "description": "Classification label data",
                        "type": ["integer", "string"]
                    }
                ]
            }
        },
        'minItems': 1
    }

    METADATA_SCHEMA = {
        "definitions": {
            "confidence": {
                "type": "number",
                "minimum": 0.0,
                "maximum": 1.0
            },
            "class-map": {
                "type": "object",
                "propertyNames": {"pattern": "^[0-9]+"},
                "patternProperties": {"": {"type": "string", "minLength": 1, "maxLength": 256}}
            },
            "objects": {
                "type": "array",
                "items": {
                    "type": "object",
                    "required": [
                        "confidence"
                    ],
                    "properties": {
                        "confidence": {
                            "$ref": "#/definitions/confidence"
                        }
                    }
                }
            }
        },
        "patternProperties": {
            "^[A-Za-z0-9-]+": {
                "description": "Ground Truth label metadata",
                "properties": {
                    "class-map": {
                        "$ref": "#/definitions/class-map"
                    },
                    "human-annotated": {
                        "type": "string",
                        "enum": ["yes", "no"]
                    },
                    "type": {
                        "type": "string",
                        "enum": ["groundtruth/image-classification", "groundtruth/object-detection"]
                    },
                    "creation-date": {
                        "type": "string",
                        "format": "date-time"
                    },
                    "confidence": {
                        "$ref": "#/definitions/confidence"
                    },
                    "class-name": {
                        "type": "string",
                        "minLength": 1,
                        "maxLength": 256
                    },
                    "objects": {
                        "$ref": "#/definitions/objects"
                    }
                },
                "required": [
                    "human-annotated",
                    "type",
                    "creation-date"
                ],
                "oneOf": [
                    {
                        "required": [
                            "confidence",
                            "class-name"
                        ]
                    },
                    {
                        "required": [
                            "class-map",
                            "objects"
                        ]
                    }
                ]
            }
        },
        'minItems': 5
    }

AUTO_ML_SERVICE_INPUT_BUCKET = "rekognition-automl-training"
AUTO_ML_SERVICE_ARTIFACTS_BUCKET = "rekognition-automl-models"
AUTO_ML_MODEL_ZOO_ARTIFACTS_BUCKET = "rekognition-automl-modelzoo"

# Lambda input event constants
ACCOUNT_ID = "AccountId"
PROJECT_ID = "ProjectId"
VERSION_NAME = "VersionName"
PROJECT_VERSION_ARN = "ProjectVersionArn"
S3_PROJECT_PREFIX = "S3ProjectVersionPrefix"
HPO_JOB_NAME_OVERWRITE = "HPOJobName"
USE_CASE = "UseCase"
EXECUTION_START_TIME = "executionStartTime"

# Lambda input context constants
REGION = "region"
SERVICE_ACCOUNT_ID = "service_account_id"
VERSION = "version"

# GT manifest fields from API definition
DATASETS_KEY = "Datasets"
USE_TYPE_KEY = "UseType"
GT_MANIFEST_KEY = "GroundTruthManifest"
S3_OBJECT_KEY = "S3Object"
TRAIN_MANIFEST_TYPE = "TRAINING"
TEST_MANIFEST_TYPE = "TESTING"
OUTPUT_CONFIG_OWNER_ID_KEY = "OutputConfigOwnerId"
TRAIN_DATASET_OWNER_ID_KEY = "TrainDatasetOwnerId"
TEST_DATASET_OWNER_ID_KEY = "TestDatasetOwnerId"

# Sagemaker GT Augmented manifest constants
SOURCE_REF = "source-ref"
ORIGINAL_SOURCE = "original-source"
METADATA_SUFFIX = "metadata"
ATTRIBUTE_ANNOTATIONS = "annotations"
ATTRIBUTE_IMAGE_SIZE = "image_size"
ATTRIBUTE_IMAGE_WIDTH = "width"
ATTRIBUTE_IMAGE_HEIGHT = "height"
ATTRIBUTE_IMAGE_LEFT = "left"
ATTRIBUTE_IMAGE_TOP = "top"
METADATA_CLASS_MAP = "class-map"
METADATA_CLASS_NAME = "class-name"
METADATA_CLASS_ID = "class_id"
METADATA_OBJECTS = "objects"
METADATA_HUMAN_ANNOTATED = "human-annotated"
METADATA_CONFIDENCE = "confidence"
METADATA_JOB_NAME = "job-name"
METADATA_TYPE = "type"
METADATA_CREATION_DATE = "creation-date"
LABEL_TYPE_CLASSIFICATION = "image-classification"
LABEL_TYPE_DETECTION = "object-detection"
GROUND_TRUTH_TYPE_PREFIX = "groundtruth/"

# Intermediate SM training manifest constants
LABEL = "label"  # validated output attribute for SM training
ANNOTATION_TYPE = "annotation-type"  # use type for SM training
LINE_INDEX = "line-idx"  # line number from original manifest
IMAGE_RESIZE_RATIO = "resize-ratio"

# Output GT manifest constants
AUTOML_OBJECT_ID = "rekognition-custom-labels-object-id"
AUTOML_JOB_NAME = "rekognition-custom-labels-training-job"
AUTOML_LABEL = "rekognition-custom-labels-training"

# File constants
JSON_EXT = ".json"
CSV_EXT = ".csv"
UTF_8 = "utf-8"

# Manifest file
# Manifest format reference: https://quip-amazon.com/tkHTAzxM91P7/AutoML-Manifest-Format
TRAINING_MANIFEST = TRAIN_MANIFEST_TYPE + JSON_EXT
TESTING_MANIFEST = TEST_MANIFEST_TYPE + JSON_EXT
VALIDATED_GT_MANIFEST_SUFFIX = "_gt"  # validated GT-compatible manifest for customer
VALIDATED_MANIFEST_STATS_SUFFIX = '_stats'  # stats required for technique selection and evaluation
VALIDATED_MANIFEST_STATS_AFTER_FILTER_SUFFIX = '_stats_after_filter'  # stats after filter dataset
PRESPLIT_MANIFEST = "_presplit"  # original training manifest (before auto-split)
SM_MANIFEST = "SageMakerManifest"
GT_MANIFEST = "GroundTruthManifest"
VALID_SUFFIX = "_valid"  # suffix for manifest files after passing initial syntax and image validation checks
SPLIT_SUFFIX = "_split"  # suffix for manifest files after auto-split step. Does not imply if dataset was autosplit
MIN_DATASET_CLASSES = 2
MAX_DATASET_CLASSES = 250

# S3 constants
S3_PREFIX = "s3://"
CONTENT_LENGTH = "ContentLength"
CHUNK_SIZE_IN_BYTES = 200*1024*1024  # batch size for streaming s3 download
MIN_MULTIPART_UPLOAD_CHUNK_SIZE = 5*1024*1024  # 5MB
MAX_MULTIPART_UPLOAD_CHUNK_SIZE = 5*1024*1024*1024  # 5GB
MOCK_S3_BUCKET_OWNER_ID = '75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c078efc7c6caea54ba06a'  # This is fixed owner-id
# returned by mock_s3 used in unit tests.

# Validation constants
MAX_DATASET_OBJECTS = 250000
ONE_GB_IN_BYTES = 1*1024*1024*1024
MAX_MANIFEST_FILE_SIZE_IN_BYTES = ONE_GB_IN_BYTES
MAX_IMAGE_SIZE_IN_BYTES = 15*1024*1024  # 15MB
MAX_IMAGE_DIMENSION = 4096  # max supported image size
MIN_IMAGE_DIMENSION = 64  # min supported image size
RESIZE_IMAGE_DIMENSION = 1280  # image dimension re-size threshold
MAX_RESIZED_IMAGE_IN_BYTES = RESIZE_IMAGE_DIMENSION * RESIZE_IMAGE_DIMENSION * 3  # 1-byte per plane for RGB
S3_VERSION = "Version"
JPEG_QUALITY = 100

# Training data related constants
TEST_SPLIT_RATIO = 0.2
MIN_EXPECTED_NUM_CLASS_LABELS = 2
MAX_BBS_PER_IMAGE = 50
MAX_DATASET_ERROR_THRESHOLD_PERCENT = 20  # percent of invalid dataset objects/images which will cause job failure
PERCENT_100 = 100
MIN_TEST_TRAIN_OVERLAP_PERCENT = 50  # minimum overlap percent between test and train labels/classes
MIN_BB_DIMENSION = 1.0  # lines with BB width/height smaller than this will be skipped

# S3 Batch constants
BATCH_MANIFEST_FORMAT = "S3BatchOperations_CSV_20180820"
BATCH_REPORT_FORMAT = "Report_CSV_20180820"
BATCH_COPY_PREFIX = "/copy"  # output of batch copy
BATCH_VALIDATE_PREFIX = "/image_validation"  # output of image validation
BATCH_REPORT_PREFIX = "/report"
BATCH_REPORT_MANIFEST_JSON = "manifest.json"
BATCH_REPORT_JOB_PREFIX = "job"
TASK_SUCCEEDED = "succeeded"
TASK_FAILED = "failed"

# State machine result keys
BATCH_COPY_RESULT = "BatchCopyResult"
BATCH_COPY_JOBS = "BatchCopyJobIds"
IMAGE_VALIDATION_RESULT = "ImageValidationResult"
IMAGE_VALIDATION_JOBS = "ImageValidationJobIds"
INPUT_DATASET_SIZE = "DatasetSize"  # total input dataset size (includes invalid lines/images)
VALID_TRAINING_DATASET_SIZE = "ValidTrainingDatasetSize"  # valid images in training set
VALID_TESTING_DATASET_SIZE = "ValidTestingDatasetSize"  # valid images in testing set

# Image validation lambda task status
IMAGE_VALIDATION_PERMANENT_FAILURE = "PermanentFailure"
IMAGE_VALIDATION_SUCCESS = "Succeeded"

# Lambda constants
IMAGE_VALIDATION_FUNCTION = "ImageValidationFunction"

# Create Model Artifacts Constants
MODEL_GRAPH_DICTIONARY = "ModelGraphDictionary"
MODEL_ZOO_VERSION = "model_zoo_version"

# Model Zoo constants for techniques and use cases
MZOO_TECHNIQUE_FINETUNING = "FINETUNING"
MZOO_TECHNIQUE_MULTISTAGE = "MULTISTAGE"
MZOO_TECHNIQUE_INSTANCE_MATCH = "INSTANCEMATCH"
MZOO_USE_CASE_CLASSIFICATION = "CLASSIFICATION"
MZOO_USE_CASE_DETECTION = "DETECTION"

# Hardcode training instance type for now
TRAINING_INSTANCE_TYPE = "ml.p3.2xlarge"

# workflow execution output
WORKFLOW_OUTPUT = "WorkflowOutput"

# keys in workflow execution output
JOB_STATUS = "jobStatus"
CHILD_WORKFLOW_EXECUTION_STARTTIME = "childWorkflowExecutionStartTime"
WORKFLOW_NAME = "workflowName"

# StepFunction Job Status
JOB_STATUS_SUCCEED = "SUCCEED"
JOB_STATUS_CLIENT_ERROR = "CLIENT_ERROR"
JOB_STATUS_SERVICE_ERROR = "SERVICE_ERROR"

# Autosplit lambda response keys
IS_AUTOSPLIT_KEY = 'IS_AUTOSPLIT'

In [132]:
import logging
import jsonlines
from jsonlines.jsonlines import InvalidLineError
import fastjsonschema
#from datetime import datetime

#from rekognition_auto_ml_lambda.schemas.schema import Schemas
#from rekognition_auto_ml_lambda.utils.constants import SOURCE_REF, METADATA_SUFFIX, LABEL
#from rekognition_auto_ml_lambda.utils.constants import ATTRIBUTE_ANNOTATIONS, ATTRIBUTE_IMAGE_SIZE
#from rekognition_auto_ml_lambda.utils.constants import METADATA_CLASS_MAP, METADATA_CLASS_NAME, \
#    METADATA_HUMAN_ANNOTATED, METADATA_OBJECTS, METADATA_CONFIDENCE, ANNOTATION_TYPE, LABEL_TYPE_CLASSIFICATION, \
#    LABEL_TYPE_DETECTION, LINE_INDEX, METADATA_TYPE, METADATA_CREATION_DATE, METADATA_CLASS_ID

LOGGER = logging.getLogger(__name__)


class GroundTruthManifestStatistics:
    """
    Manifest statistics
    """
    def __init__(self):
        self.total_line_count = 0
        self.source_ref_only_line_count = 0
        self.valid_line_count = 0
        self.valid_annotations = 0
        self.invalid_annotations = 0

    def __add__(self, stats):
        self.total_line_count += stats.total_line_count
        self.source_ref_only_line_count += stats.source_ref_only_line_count
        self.valid_line_count += stats.valid_line_count
        self.valid_annotations += stats.valid_annotations
        self.invalid_annotations += stats.invalid_annotations
        return self

    def get_total_line_without_source_ref_only_line(self):
        # do not include line with image but no annotation in total
        # https://sim.amazon.com/issues/REKN-3195
        return self.total_line_count - self.source_ref_only_line_count

    def __eq__(self, stats):
        try:
            return self.total_line_count == stats.total_line_count \
                and self.source_ref_only_line_count == self.source_ref_only_line_count \
                and self.valid_line_count == stats.valid_line_count \
                and self.valid_annotations == stats.valid_annotations \
                and self.invalid_annotations == stats.invalid_annotations
        except AttributeError:
            return False

    def __str__(self):
        return "total line:{0}, source_ref_only:{1}, valid_line:{2}, valid_annotations:{3}, invalid_annotations:{4}"\
            .format(self.total_line_count, self.source_ref_only_line_count, self.valid_line_count,
                    self.valid_annotations, self.invalid_annotations)


class GroundTruthManifestValidator:
    """
    Manifest validation class
    """

    def __init__(self):

        self.source_validator = fastjsonschema.compile(Schemas.SOURCE_SCHEMA)
        self.metadata_validator = fastjsonschema.compile(Schemas.METADATA_SCHEMA)
        self.attribute_validator = fastjsonschema.compile(Schemas.ATTRIBUTE_SCHEMA)

    def _transform_attribute_and_metadata(self, attribute, metadata):
        """Extract necessary fields
        Currently we only retain fields which can fit into OpenImage format used by science training code

        :param attribute: attribute info
        :param metadata: metadata info
        :return: validated label
        """

        valid_label = {}
        if LABEL_TYPE_CLASSIFICATION in metadata[METADATA_TYPE]:
            # classification
            valid_label[METADATA_CLASS_NAME] = metadata[METADATA_CLASS_NAME]
            valid_label[METADATA_CONFIDENCE] = metadata[METADATA_CONFIDENCE]
            valid_label[METADATA_HUMAN_ANNOTATED] = metadata[METADATA_HUMAN_ANNOTATED]
            valid_label[METADATA_CREATION_DATE] = metadata[METADATA_CREATION_DATE]
            valid_label[ANNOTATION_TYPE] = LABEL_TYPE_CLASSIFICATION
        else:
            # detection
            valid_label[METADATA_HUMAN_ANNOTATED] = metadata[METADATA_HUMAN_ANNOTATED]
            valid_label[METADATA_OBJECTS] = metadata[METADATA_OBJECTS]
            valid_label[METADATA_CLASS_MAP] = metadata[METADATA_CLASS_MAP]
            valid_label[ATTRIBUTE_ANNOTATIONS] = attribute[ATTRIBUTE_ANNOTATIONS]
            valid_label[ATTRIBUTE_IMAGE_SIZE] = attribute[ATTRIBUTE_IMAGE_SIZE]
            valid_label[METADATA_CREATION_DATE] = metadata[METADATA_CREATION_DATE]
            valid_label[ANNOTATION_TYPE] = LABEL_TYPE_DETECTION

        return valid_label

    def _perform_deep_annotation_check(self, attribute, metadata, current_manifest_stats, line_number):
        """Perform deep checks across attribute and its metadata

        :param annotation: annotation
        :return: annotation if valid, None if invalid
        """
        annotation = self._transform_attribute_and_metadata(attribute, metadata)
        if LABEL_TYPE_DETECTION in annotation[ANNOTATION_TYPE]:
            # Validate if every bounding box label has a confidence
            if len(annotation[ATTRIBUTE_ANNOTATIONS]) != len(annotation[METADATA_OBJECTS]):
                current_manifest_stats.invalid_annotations += 1
                LOGGER.info("line %s: Ground truth annotation does not contain valid confidence objects", line_number)
                print(f"line {line_number}: Ground truth annotation does not contain valid confidence objects" )
                return None

            # Validate if bounding box class_id exists in the class map
            for bounding_box in annotation[ATTRIBUTE_ANNOTATIONS]:
                if str(bounding_box[METADATA_CLASS_ID]) not in annotation[METADATA_CLASS_MAP]:
                    current_manifest_stats.invalid_annotations += 1
                    LOGGER.info("line %s: Ground truth annotation does not contain valid class_id: %s",
                                line_number, str(bounding_box[METADATA_CLASS_ID]))
                    print(f"line {line_number}: Ground truth annotation does not contain valid class_id: {str(bounding_box[METADATA_CLASS_ID])}")
                    return None

        return annotation

    def _is_valid_source_ref(self, line, line_number):
        """
        Validate if this line contains a valid source-ref value that matches specified JSONSchema
        """
        if SOURCE_REF not in line:
            LOGGER.info("line %s: Ground truth annotation does not contain valid image source reference", line_number)
            print(f"line {line_number}: Ground truth annotation does not contain valid image source reference")
            return False
        try:
            self.source_validator({SOURCE_REF: line[SOURCE_REF]})
        except fastjsonschema.exceptions.JsonSchemaException:
            LOGGER.info("line %s: Invalid image source reference format", line_number)
            print(f"line {line_number}: Invalid image source reference format")
            return False
        return True

    def _get_attributes_from_line(self, line, line_number):
        """Validate each GT manifest line entry and create list of annotations

        :param line: GT manifest JSON line
        :return: Returns image source and list of valid annotations
        """
        # Filter attributes which have metadata
        attributes = {k: v for k, v in line.items() if f'{k}-{METADATA_SUFFIX}' in line}
        if not attributes:
            LOGGER.info("line %s: Ground truth annotation does not contain any valid attributes", line_number)
            print(f"line {line_number}: Ground truth annotation does not contain any valid attributes")
            return None
        return attributes

    def _source_ref_only(self, line):
        # _is_valid_source_ref function already verified the key exist
        return len(line.keys()) == 1

    def _validate_line(self, line, attributes, line_number, current_manifest_stats):
        # Validate attribute and metadata against schema and populate necessary fields
        valid_annotations = []
        for key, value in attributes.items():
            metadata_key = f'{key}-{METADATA_SUFFIX}'
            try:
                self.attribute_validator({key: value})
                self.metadata_validator({metadata_key: line[metadata_key]})
                valid_annotation = self._perform_deep_annotation_check(
                    line[key], line[metadata_key], current_manifest_stats, line_number)
                if valid_annotation:
                    valid_annotations.append(valid_annotation)
                    current_manifest_stats.valid_annotations += 1
            except fastjsonschema.exceptions.JsonSchemaException as exc:
                current_manifest_stats.invalid_annotations += 1
                LOGGER.info("line %s: Invalid annotation for attribute", line_number)
                print(f"line {line_number}: Invalid annotation for attribute")
                LOGGER.debug("line %s: Invalid annotation for attribute: %s due to: %s", line_number, key, exc)
                print(f"line {line_number}: Invalid annotation for attribute: {key} due to: {exc}")

        return {SOURCE_REF: line[SOURCE_REF], LABEL: valid_annotations, LINE_INDEX: line_number} if valid_annotations \
            else None

    def validate_manifest(self, manifest, existing_manifest_stats):
        """

        :param manifest: manifest file
        :param existing_manifest_stats: contains the statistics of existing
        manifest file
        :return: manifest and stats
        """
        sanitized_manifest = []
        line_number = existing_manifest_stats.total_line_count + 1
        current_manifest_stats = GroundTruthManifestStatistics()
        reader = jsonlines.Reader(manifest)
        while True:
            line = None
            try:
                line = reader.read(type=dict)
            except (InvalidLineError, UnicodeDecodeError) as exc:
                LOGGER.debug("line %s: Unexpected jsonline error in manifest: %s", line_number, exc)
                print(f"line {line_number}: Unexpected jsonline error in manifest: {exc}")
            except EOFError:
                LOGGER.info("line %s: Finished parsing manifest file (not all are necessarily valid)", line_number)
                print(f"line {line_number}: Finished parsing manifest file (not all are necessarily valid)")
                break

            if line and self._is_valid_source_ref(line, line_number):
                attributes = self._get_attributes_from_line(line, line_number)
                if attributes:
                    validated_line = self._validate_line(line, attributes,
                                                         line_number, current_manifest_stats)
                    if validated_line:
                        sanitized_manifest.append(validated_line)
                        current_manifest_stats.valid_line_count += 1
                elif self._source_ref_only(line):
                    current_manifest_stats.source_ref_only_line_count += 1

            line_number += 1
            current_manifest_stats.total_line_count += 1

        return sanitized_manifest, current_manifest_stats + existing_manifest_stats

In [138]:
stats = GroundTruthManifestStatistics()
validator = GroundTruthManifestValidator()
manif, stats2 = validator.validate_manifest(open('data/train_manifest_5_limit.json', 'r'), stats)

line 66: Ground truth annotation does not contain valid class_id: 0
line 219: Ground truth annotation does not contain valid class_id: 0
line 258: Ground truth annotation does not contain valid class_id: 0
line 283: Finished parsing manifest file (not all are necessarily valid)


In [139]:
print(stats2)

total line:282, source_ref_only:0, valid_line:279, valid_annotations:279, invalid_annotations:3


### Saving Manifest in S3

In [141]:
s3.upload_file(f'data/train_manifest{manifest_ext}.json', bucket, f"{reko_manifest_path}/train_manifest{manifest_ext}.json")
s3.upload_file(f'data/test_manifest{manifest_ext}.json', bucket, f"{reko_manifest_path}/test_manifest{manifest_ext}.json")

In [50]:
reko = boto3.client("rekognition")

In [57]:
reko.describe_projects()['ProjectDescriptions']

[{'ProjectArn': 'arn:aws:rekognition:us-east-1:113147044314:project/construction-object-recognition/1582735151793',
  'CreationTimestamp': datetime.datetime(2020, 2, 26, 16, 39, 11, 793000, tzinfo=tzlocal()),
  'Status': 'CREATED'},
 {'ProjectArn': 'arn:aws:rekognition:us-east-1:113147044314:project/vtg-demo-1/1579815781951',
  'CreationTimestamp': datetime.datetime(2020, 1, 23, 21, 43, 1, 951000, tzinfo=tzlocal()),
  'Status': 'CREATED'}]

In [62]:
def get_project(project_name="construction-object-recognition"):
    try:
        project = reko.create_project(ProjectName=project_name)['ProjectArn']
    except reko.exceptions.ResourceInUseException:
        project = 'arn:aws:rekognition:us-east-1:113147044314:project/construction-object-recognition/1582735151793'
    return(project)

In [63]:
project = get_project()

In [65]:
reko.describe_project_versions(ProjectArn=project)['ProjectVersionDescriptions']

[{'ProjectVersionArn': 'arn:aws:rekognition:us-east-1:113147044314:project/construction-object-recognition/version/construction-object-recognition.2020-02-27T01.20.12/1582762812913',
  'CreationTimestamp': datetime.datetime(2020, 2, 27, 0, 20, 12, 913000, tzinfo=tzlocal()),
  'Status': 'TRAINING_IN_PROGRESS',
  'StatusMessage': 'The model is being trained.',
  'OutputConfig': {'S3Bucket': 'custom-labels-console-us-east-1-0435516b92',
   'S3KeyPrefix': 'evaluation/arn:aws:rekognition:us-east-1:113147044314:project/construction-object-recognition/1582735151793'},
  'TrainingDataResult': {'Input': {'Assets': [{'GroundTruthManifest': {'S3Object': {'Bucket': 'custom-labels-console-us-east-1-0435516b92',
        'Name': 'datasets/construction-objects-train/manifests/output/output.manifest',
        'Version': 'CiD5meCOk_kuud5Eti4c8IDavb1sw7D1'}}}]}},
  'TestingDataResult': {'Input': {'Assets': [], 'AutoCreate': True}}},
 {'ProjectVersionArn': 'arn:aws:rekognition:us-east-1:113147044314:proje